In [ ]:
import requests
import json
import os
import cv2
from PIL import Image
import shutil
import pandas as pd

names = ['aphanizomenon', 'detritus', 'dolichospermum', 'microcystis', 'oscillatoria', 'water bubble', 'woronichinia']

In [ ]:
def retrieve_existing_labels(base = 'images', export = 'new_images', existing = 'b_ds'):
    if not os.path.exists(export):
        os.makedirs(export)
    else:
        shutil.rmtree(export)
        os.makedirs(export)
    found = 0
    
    for genus in os.listdir(base):
        if genus == ".DS_Store" or genus == 'backgrounds':
            continue
        
        if not os.path.exists(f"{export}/{genus}"):
                os.makedirs(f"{export}/{genus}")
        else:
            shutil.rmtree(f"{export}/{genus}")
            os.makedirs(f"{export}/{genus}")

        if not os.path.exists(f"{export}/{genus}/images"):
            os.makedirs(f"{export}/{genus}/images")
        else:
            shutil.rmtree(f"{export}/{genus}/images")
            os.makedirs(f"{export}/{genus}/images")
        
        if not os.path.exists(f"{export}/{genus}/labels"):
            os.makedirs(f"{export}/{genus}/labels")
        else:
            shutil.rmtree(f"{export}/{genus}/labels")
            os.makedirs(f"{export}/{genus}/labels")
        for image in os.listdir(f"{base}/{genus}"):
            image_name, image_ext = os.path.splitext(image)
            # if folder images and labels don't exist, create them
            
            
            # copy image to images folder
            shutil.copy(f"{base}/{genus}/{image}", f"{export}/{genus}/images/{image}")

            
            for split in ['train', 'valid', 'test']:
                for label in os.listdir(f'{existing}/{split}/labels'):
                    label_name, label_ext = os.path.splitext(label)

                    if label == ".DS_Store":
                        continue
                    try:
                        if label_name in image_name:
                            # normal picture
                            shutil.copy(f"{existing}/{split}/labels/{label}", f"{export}/{genus}/labels/{label}")
                    except:
                        if label_name in image_name:
                            # is a lab picture
                            shutil.copy(f"{existing}/{split}/labels/{label}", f"{export}/{genus}/labels/{image_name}.txt")
                            found += 1
            
    print("found", found)

In [ ]:
retrieve_existing_labels(base = 'images', export = 'new_images', existing = 'b_ds')

In [ ]:
# get all images from images folder and put it into a set
images_set = set()
for c in os.listdir("images"):
    if c == ".DS_Store":
        continue
    for image in os.listdir(f"images/{c}"):
        name, ext = os.path.splitext(image)
        images_set.add(name)
print(images_set)

In [ ]:
x = 0
l = 0
currset = set([])
for genus in os.listdir('images'):
    if genus == ".DS_Store":
        continue

    for image in os.listdir(f"images/{genus}"):
        n, ext = os.path.splitext(image)
        print(n)
        if n in currset:
            print('currset', n)
            continue
        if n in cannot_use:
            print(n)
            x+=1
        else:
            l +=1
        
        currset.add(n)
print(x)
print(l)

In [ ]:
print(len(os.listdir('images/oscillatoria')))

In [ ]:
# 17802507, 
for file in os.listdir('images/aphanizomenon'):
    name, ext = os.path.splitext(file)
    print(name)


In [ ]:
print(licenses.keys())

# get unique authors and their corresponding images
authors = {}

for license_type in licenses:
    for author in licenses[license_type]:
        if author not in authors:
            authors[author] = []
        
        for image, image_link in licenses[license_type][author].items():
            authors[author].append(image)


for author in authors:
    print(author)
    print(len(authors[author]))


# send authors dictionary to json file
with open('authors.json', 'w') as f:
    json.dump(authors, f)


In [ ]:
x = requests.get(f"https://api.inaturalist.org/v1/observations?photos=true&q=Microcystis&search_on=names&quality_grade=research&order=desc&order_by=created_at&per_page=100&page=1")
x = x.json()

ct = int(x['total_results']) # counter for number of images to collect


count = 1
print(len(x['results']))

for observation in x['results']:
    photos = observation['photos']
    for i in range(len(photos)):
        photo = photos[i]
        url = photos[i]['url']
        # res[i] = res[i].index("square")
        print(photo)
        

In [ ]:
print(len(images_set))

In [ ]:
# create copy of b_ds folder
if os.path.exists("b_ds_need_label"):
    shutil.rmtree("b_ds_need_label")

shutil.copytree("b_ds", "b_ds_need_label")

In [ ]:
# send all water bubble and detritus images to images/water bubble and images/detritus
for c in os.listdir("b_ds"):
    if c == "test" or c == "train" or c == "valid":
        for image in os.listdir(f"b_ds/{c}/images"):
            name, ext = os.path.splitext(image)
            if 'WaterBubble' in name:
                shutil.copy(f"b_ds/{c}/images/{image}", f"images/water_bubble/{image}")
            elif 'Detritus' in name:
                shutil.copy(f"b_ds/{c}/images/{image}", f"images/detritus/{image}")

In [ ]:
print(len(os.listdir("images/water_bubble")))
print(len(os.listdir("images/detritus")))

In [ ]:
print(len(os.listdir('images/aphanizomenon')) + len(os.listdir('images/dolichospermum')) + len(os.listdir('images/microcystis')) + len(os.listdir('images/oscillatoria')) + len(os.listdir('images/woronichinia')))

print('NP_WaterBubble_075.jpg' in os.listdir('images/aphanizomenon'))
print('NP_WaterBubble_075.jpg' in os.listdir('images/dolichospermum'))
print('NP_WaterBubble_075.jpg' in os.listdir('images/microcystis'))
print('NP_WaterBubble_075.jpg' in os.listdir('images/oscillatoria'))
print('NP_WaterBubble_075.jpg' in os.listdir('images/woronichinia'))

In [ ]:
#

In [ ]:

# check for intersection 
x = 0
actual_set = set()
for split in os.listdir('b_ds'):
    if split == 'test' or split == 'train' or split == 'valid':
        print(split)
        for image in os.listdir(f'b_ds/{split}/images'):
            # rename image at this path to remove "_jpg"
            # get index of "_jpg"
            name, ext = os.path.splitext(image)
            # if it has not been labeled yet, add it to the set
            if name in images_set or 'WaterBubble' in name or 'Detritus' in name:
                x += 1
                actual_set.add(image)

                # TODO: add corresponding label to labels folder
                
            else:
                # send to testing folder
                # print(image)
                pass

print(x)

In [ ]:
print(actual_set)

In [ ]:
# 18-08-20-AGL-DEP-SRCP-microcystis_jpg.rf.7441f86b59469b8d204133277d98871d.jpg
s = "18-08-20-AGL-DEP-SRCP-microcystis_jpg.rf.7441f86b59469b8d204133277d98871d.jpg"
index = s.find("_jpg")


In [ ]:
# clean up roboflow imported dataset
def clean_roboflow_dataset():
    # get index of "_jpg" 
    for split in os.listdir('b_ds'):
        if split == 'test' or split == 'train' or split == 'valid':
            print(split)
            for image in os.listdir(f'b_ds/{split}/images'):
                # rename image at this path to remove "_jpg"
                # get index of "_jpg"
                name, ext = os.path.splitext(image)
                new_image = image[:image.rfind('_')] + ext
                # replace current image with new image
                # get image path

                os.rename(f'b_ds/{split}/images/{image}', f'b_ds/{split}/images/{new_image}')
            
            for label in os.listdir(f'b_ds/{split}/labels'):
                # rename label at this path to remove "_txt"
                # get index of "_txt"
                name, ext = os.path.splitext(label)
                new_label = label[:label.rfind('_')] + ext
                
                os.rename(f'b_ds/{split}/labels/{label}', f'b_ds/{split}/labels/{new_label}')

clean_roboflow_dataset()

In [ ]:
names = ['aphanizomenon', 'detritus', 'dolichospermum', 'microcystis', 'oscillatoria', 'water bubble', 'woronichinia']

In [ ]:
def get_taxon_test(taxon, page=1, licenses={}, authors = {}):
    
    orig_x = requests.get(f"https://api.inaturalist.org/v1/observations?photos=true&taxon_name={taxon}&quality_grade=research&per_page=100&page={page}").json()
    x = orig_x['results']
    print(len(x))
    print(orig_x['total_results'])

    if not os.path.exists(f'images/{taxon}'):
        os.makedirs(f'images/{taxon}')

    else:
        # destroy existing folder
        shutil.rmtree(f'images/{taxon}')
        os.makedirs(f'images/{taxon}')

    curr_obs = orig_x['total_results']

    if curr_obs > 0 and len(x) > 0:
        for observation in x:
            photos = observation['photos']
            for i in range(len(photos)):
                url = photos[i]['url']
                photo = photos[i]
                
                licenses[photo['license_code']] = licenses.get(photo['license_code'], {})
                print(licenses[photo['license_code']])

                licenses[photo['license_code']][photo['attribution']] = licenses[photo['license_code']].get(photo['attribution'], {})

                # store image and path under each author
                licenses[photo['license_code']][photo['attribution']][f'{taxon}-{url.split("/")[-2]}-{url.split("/")[-1]}'] = url

                
                file = open(f'images/{taxon}/{taxon}-{url.split("/")[-2]}-{url.split("/")[-1]}', 'wb')
                file.write(requests.get(url).content)
                file.close()
                

            curr_obs-=1

        print(licenses)
        print(authors)

        get_taxon_test(taxon, page+1, licenses, authors)

In [ ]:
print(len(os.listdir('images/microcystis')))

In [ ]:
# add background images - all fetched from local storage

def add_backgrounds():
    # get from Bacteria Dataset
    failed_count = 0
    for genus in os.listdir('Bacteria Dataset/Dataset-1'):
        if genus == '.DS_Store' or (genus in names):
            continue

        for img in os.listdir(f'Bacteria Dataset/Dataset-1/{genus}'):
            if img == '.DS_Store':
                continue
            # move image to respective folders
            name, ext = os.path.splitext(img)
            
            im = Image.open(f'Bacteria Dataset/Dataset-1/{genus}/{img}')
            
            try:
                rgb_im = im.convert('RGB')
                rgb_im.save(f'images/backgrounds/{name}_background.jpg')
            except:
                failed_count+=1

    
    for genus in os.listdir('Bacteria Dataset/Dataset-2'):
        if genus == '.DS_Store' or (genus in names):
            continue
        
        for img in os.listdir(f'Bacteria Dataset/Dataset-2/{genus}'):
            if img == '.DS_Store':
                continue
            # move image to respective folders
            name, ext = os.path.splitext(img)

            im = Image.open(f'Bacteria Dataset/Dataset-2/{genus}/{img}')
            
            try:
                rgb_im = im.convert('RGB')
                rgb_im.save(f'images/backgrounds/{name}_background.jpg')
            except:
                failed_count+=1
    

    
    print('failed conversion of', failed_count)

In [ ]:
# get total number of items
# size: square, medium, large
def get_taxon(taxon, licenses = {}, useNums = False, base = 'images', remove = True):
    # revert back to before and compare images to see if they are the same
    page=1
    x = requests.get(f"https://api.inaturalist.org/v1/observations?photos=true&taxon_name={taxon}&quality_grade=research&per_page=100&page={page}")
    x = x.json()

    ct = int(x['total_results']) # counter for number of images to collect
    if remove:
        if not os.path.exists(f'{base}/{taxon}'):
            os.makedirs(f'{base}/{taxon}')

        else:
            # destroy existing folder
            shutil.rmtree(f'{base}/{taxon}')
            os.makedirs(f'{base}/{taxon}')

    
    count = 1
    while len(x['results']) != 0:
        print(len(x['results']))
        for observation in x['results']:
            photos = observation['photos']
            for i in range(len(photos)):
                photo = photos[i]
                url = photos[i]['url']
                # res[i] = res[i].index("square")

                url = url.replace("square", "original")
                
                name, extension = os.path.splitext(url)

                
                if photo['license_code'] != None:
                    response = requests.get(url, stream=True)
                    if not useNums:
                        with open(f'{base}/{taxon}/{taxon}-{url.split("/")[-2]}-{url.split("/")[-1]}', 'wb') as out_file:
                            shutil.copyfileobj(response.raw, out_file)
                        del response
                    else:
                        with open(f'{base}/{taxon}/{taxon}-{count}{extension}', 'wb') as out_file:
                            shutil.copyfileobj(response.raw, out_file)
                        del response
                        count+=1
                    
                    # convert image to RGB if it is not
                    # check if the current photo is not jpg

                    
                    # rgb_im.save(f'finalimages/{taxon}/{taxon}-{url.split("/")[-2]}-{url.split("/")[-1]}')
                
                # store authors under each license

                licenses[photo['license_code']] = licenses.get(photo['license_code'], {})

                licenses[photo['license_code']][photo['attribution']] = licenses[photo['license_code']].get(photo['attribution'], {})

                # store image and path under each author
                licenses[photo['license_code']][photo['attribution']][f'{taxon}-{url.split("/")[-2]}-{url.split("/")[-1]}'] = url

            ct-=1
        # get the next page
        page+=1
        x = requests.get(f"https://api.inaturalist.org/v1/observations?photos=true&q={taxon}&search_on=names&quality_grade=research&order=desc&order_by=created_at&per_page=100&page={page}")
        x = x.json()
    
    return licenses

    

In [ ]:
get_taxon('woronichinia', base='test')

In [ ]:
# for f in os.listdir('test/oscillatoria'):
#     if not f in os.listdir('images/oscillatoria'):
#         print(f)

# for f in os.listdir('images/oscillatoria'):
#     if not f in os.listdir('test/oscillatoria'):
#         print(f)

# for f in os.listdir('images/aphanizomenon'):
#     if not f in os.listdir('test/aphanizomenon'):
#         print(f)


for f in os.listdir('images/woronichinia'):
    if not f in os.listdir('test/woronichinia'):
        print(f)

In [ ]:
print(len(os.listdir('images/microcystis')))

In [ ]:
# rename every file to the name of its encompassing folder
def rename_files():
    for species_name in os.listdir('images'):
        path = f'images/{species_name}'
        for filename in os.listdir(path):
            os.rename(f'{path}/{filename}', f'{path}/{species_name}-{filename}')

In [ ]:
rename_files()

In [ ]:
s = "microcystis"

for species in s.split(" "):
    print(species)
    get_species(species, "original")

In [ ]:
def get_primary_images():
    # retrieve online images

    # remove all images from images folder
    for genus in os.listdir('images'):
        if genus == '.DS_Store':
            continue
        shutil.rmtree(f'images/{genus}')
        os.makedirs(f'images/{genus}')

    # get images from b_ds
    for c in os.listdir("b_ds"):
        if c == "test" or c == "train" or c == "valid":
            for image in os.listdir(f"b_ds/{c}/images"):
                name, ext = os.path.splitext(image)
                if 'WaterBubble' in name:
                    shutil.copy(f"b_ds/{c}/images/{image}", f"images/water_bubble/{image}")
                elif 'Detritus' in name:
                    shutil.copy(f"b_ds/{c}/images/{image}", f"images/detritus/{image}")
                
    # get stuff from Bacteria 


    for genus in os.listdir('Bacteria Dataset/Dataset-1'):
        if genus == '.DS_Store' or not (genus in names):
            continue

        for img in os.listdir(f'Bacteria Dataset/Dataset-1/{genus}'):
            if img == '.DS_Store':
                continue
            # move image to respective folders
            im = Image.open(f'Bacteria Dataset/Dataset-1/{genus}/{img}')
            try:
                rgb_im = im.convert('RGB')
                rgb_im.save(f'images/{genus}/{img[:-4]}_lab.jpg')
            except:
                print("failed")
                print(img)

            # shutil.copy(f'Bacteria Dataset/Dataset-1/{genus}/{img}', f'images/{genus}/{img[:-4]}_lab.jpg')

    for genus in os.listdir('Bacteria Dataset/Dataset-2'):
        if genus == '.DS_Store' or not (genus in names):
            continue
        
        for img in os.listdir(f'Bacteria Dataset/Dataset-2/{genus}'):
            if img == '.DS_Store':
                continue
            # move image to respective folders

            im = Image.open(f'Bacteria Dataset/Dataset-2/{genus}/{img}')
            try:
                rgb_im = im.convert('RGB')
                rgb_im.save(f'images/{genus}/{img[:-4]}_lab.jpg')
            except:
                print("failed")
                print(img)

        # shutil.copy(f'Bacteria Dataset/Dataset-2/{genus}/{img}', f'images/{genus}/{img[:-4]}_lab.jpg')
    add_backgrounds()

    licenses = {}
    primary_genuses = ['aphanizomenon', 'dolichospermum', 'microcystis', 'oscillatoria', 'woronichinia']
    for genus in primary_genuses:
        print(genus)
        licenses = get_taxon(genus, licenses, remove=False)

        print("num at end", len(os.listdir(f'images/{genus}')))
    
    # save licenses to file
    with open('licenses.json', 'w') as fp:
        json.dump(licenses, fp)
    


In [ ]:
add_backgrounds()

In [ ]:
get_primary_images()

In [ ]:
print(len(os.listdir('images/microcystis')))
print(len(os.listdir('images/aphanizomenon')))
print(len(os.listdir('images/dolichospermum')))
print(len(os.listdir('images/oscillatoria')))
print(len(os.listdir('images/woronichinia')))
print(len(os.listdir('images/detritus')))
print(len(os.listdir('images/water_bubble')))

print(len(os.listdir('images/microcystis')) + len(os.listdir('images/aphanizomenon')) + len(os.listdir('images/dolichospermum')) + len(os.listdir('images/oscillatoria')) + len(os.listdir('images/woronichinia')) + len(os.listdir('images/detritus')) + len(os.listdir('images/water_bubble')) + len(os.listdir('images/backgrounds')))

In [ ]:

for genus in os.listdir('Bacteria Dataset/Dataset-1'):
    if genus == '.DS_Store' or not (genus in names):
        continue

    for img in os.listdir(f'Bacteria Dataset/Dataset-1/{genus}'):
        if img == '.DS_Store':
            continue
        # move image to respective folders
        im = Image.open(f'Bacteria Dataset/Dataset-1/{genus}/{img}')
        try:
            rgb_im = im.convert('RGB')
            rgb_im.save(f'images/{genus}/{img[:-4]}_lab.jpg')
        except:
            print("failed")
            print(img)

        # shutil.copy(f'Bacteria Dataset/Dataset-1/{genus}/{img}', f'images/{genus}/{img[:-4]}_lab.jpg')

for genus in os.listdir('Bacteria Dataset/Dataset-2'):
    if genus == '.DS_Store' or not (genus in names):
        continue
    
    for img in os.listdir(f'Bacteria Dataset/Dataset-2/{genus}'):
        if img == '.DS_Store':
            continue
        # move image to respective folders

        im = Image.open(f'Bacteria Dataset/Dataset-2/{genus}/{img}')
        try:
            rgb_im = im.convert('RGB')
            rgb_im.save(f'images/{genus}/{img[:-4]}_lab.jpg')
        except:
            print("failed")
            print(img)

        # shutil.copy(f'Bacteria Dataset/Dataset-2/{genus}/{img}', f'images/{genus}/{img[:-4]}_lab.jpg')

In [ ]:
get_primary_images()

In [ ]:
get_taxon('oscillatoria', useNums=True)

In [ ]:
print(len(os.listdir('images/oscillatoria')))

In [ ]:
print(len(os.listdir('images/microcystis')))
print(len(os.listdir('images/aphanizomenon')))
print(len(os.listdir('images/dolichospermum')))
print(len(os.listdir('images/oscillatoria')))
print(len(os.listdir('images/woronichinia')))
print(len(os.listdir('images/aphanizomenon')) + len(os.listdir('images/microcystis')) + len(os.listdir('images/dolichospermum')) + len(os.listdir('images/oscillatoria')) + len(os.listdir('images/woronichinia')))

print(len(os.listdir('final_ds/train/images')) + len(os.listdir('final_ds/test/images')) + len(os.listdir('final_ds/valid/images')))

In [ ]:
add_backgrounds()

In [ ]:
# convert licenses to pandas dataframe
df = pd.DataFrame(licenses).T
df.to_csv('licenses.csv')

In [ ]:
ct = 0
for license_type in licenses.keys():
    print(license_type)
    if license_type == None:
        continue
    
    for author in licenses[license_type].keys():
        # print(author)
        for image in licenses[license_type][author].keys():
            # print(image)
            if image.split('-')[0] in names:
                ct+=1


# count bacteria dataset images


print(ct)

In [ ]:
# count number of Nones
ct = 0
for license_type in licenses.keys():
    if license_type == "null":
        for author in licenses[license_type].keys():
            for image in licenses[license_type][author].keys():
                ct+=1

print(ct)

In [ ]:
print("tolypothrix" in names)

In [ ]:
df

In [ ]:
# count how many images are in "None"
for l in licenses:
    x = 0
    for a in licenses[l]:
        x+=len(licenses[l][a])
    print(l, x)

In [ ]:
x = 0
for genus in os.listdir('images'):
    print(genus, len(os.listdir(f'images/{genus}')))
    if genus in names:
        x+=len(os.listdir(f'images/{genus}'))

print(x)

In [ ]:
# load licenses.json
with open('licenses.json') as f:
    licenses = json.load(f)

cannot_use = set([])
for l, vals in licenses.items():
    if l == 'null':
        print(licenses[l])
        for name, images in licenses[l].items():
            for img_name, url in images.items():
                cannot_use.add(img_name)

print(len(cannot_use))

In [ ]:
print(cannot_use)

In [ ]:
print(cannot_use)

# remove images that cannot be used
for genus in os.listdir('images'):
    for img in os.listdir(f'images/{genus}'):
        if img in cannot_use:
            shutil.move(f'images/{genus}/{img}', f'cannot_use/{img}')

In [ ]:
info_oscillatoria = get_taxon("oscillatoria")

In [ ]:
print(len(os.listdir('images/oscillatoria')) + len(os.listdir('images/microcystis')) + len(os.listdir('images/dolichospermum')) + len(os.listdir('images/aphanizomenon')) + len(os.listdir('images/woronichinia')))

In [ ]:
for k, v in info_oscillatoria[None].items():
    print(k)
    print(v)

In [ ]:
info_micro = get_taxon("microcystis")

In [ ]:
print(len(os.listdir('images/microcystis')))

In [ ]:
print(len(os.listdir('images/oscillatoria')))
print(len(os.listdir('org_ds/oscillatoria')))

In [ ]:
license_temp = licenses
print(license_temp)
# check if images in images/microcystis are within

In [ ]:
# CROPPING FOR NOW BUT IN THE FUTURE MAYBE USE CROPPED SEGMENTS?
# print(cv2.imread('images/Microcystis aeruginosa/2218866-medium.jpg').shape)
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

def resize_images(name):
    di = {}
    if not os.path.exists(f'images/{name}-resized'):
        os.makedirs(f'images/{name}-resized')
    for file in os.listdir(f'images/{name}'):
        print(file)
        img = cv2.imread(f'images/{name}/{file}')
        if img is None:
            di['None'] = di.get('None', 0) + 1
        else:
            image = Image.open(f'images/{name}/{file}')
            # image.save('original.jpg')
            # crop image to a square
            width, height = image.size
            # resize to the smallest dimension out of width & height to maintain square
            image = crop_center(image, min(width, height), min(width, height))
            rgb_im = image.convert('RGB')
            # image = image.resize((416, 416))

            # save the resized image
            file = file.split("-")[0] + ".jpg"
            rgb_im.save(f'images/{name}-resized/{file}')
            # break
